![Cartoon of telecom customers](IMG_8811.png)


The telecommunications (telecom) sector in India is rapidly changing, with more and more telecom businesses being created and many customers deciding to switch between providers. "Churn" refers to the process where customers or subscribers stop using a company's services or products. Understanding the factors that influence keeping a customer as a client in predicting churn is crucial for telecom companies to enhance their service quality and customer satisfaction. As the data scientist on this project, you aim to explore the intricate dynamics of customer behavior and demographics in the Indian telecom sector in predicting customer churn, utilizing two comprehensive datasets from four major telecom partners: Airtel, Reliance Jio, Vodafone, and BSNL:

- `telecom_demographics.csv` contains information related to Indian customer demographics:

| Variable             | Description                                      |
|----------------------|--------------------------------------------------|
| `customer_id `         | Unique identifier for each customer.             |
| `telecom_partner `     | The telecom partner associated with the customer.|
| `gender `              | The gender of the customer.                      |
| `age `                 | The age of the customer.                         |
| `state`                | The Indian state in which the customer is located.|
| `city`                 | The city in which the customer is located.       |
| `pincode`              | The pincode of the customer's location.          |
| `registration_event` | When the customer registered with the telecom partner.|
| `num_dependents`      | The number of dependents (e.g., children) the customer has.|
| `estimated_salary`     | The customer's estimated salary.                 |

- `telecom_usage` contains information about the usage patterns of Indian customers:

| Variable   | Description                                                  |
|------------|--------------------------------------------------------------|
| `customer_id` | Unique identifier for each customer.                         |
| `calls_made` | The number of calls made by the customer.                    |
| `sms_sent`   | The number of SMS messages sent by the customer.             |
| `data_used`  | The amount of data used by the customer.                     |
| `churn`    | Binary variable indicating whether the customer has churned or not (1 = churned, 0 = not churned).|


In [2]:
# Import libraries and methods/functions
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import RidgeClassifier, LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix

# Start your code here!

In [3]:
import pandas as pd

try:
    df1 = pd.read_csv('telecom_demographics.csv')
    df2 = pd.read_csv('telecom_usage.csv')
except FileNotFoundError as e:
    print(f"Error: {e}")

    df1 = pd.DataFrame()
    df2 = pd.DataFrame()

if not df1.empty and not df2.empty:
    churn_df = pd.merge(df1, df2)

    churn_rate = churn_df['churn'].mean()

    categorical_vars = churn_df.select_dtypes(include=['object', 'category']).columns.tolist()

    churn_rate, categorical_vars
else:
    print("One or both of the DataFrames are empty. Please check the CSV files.")


In [4]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import pandas as pd

encoder = OneHotEncoder(sparse=False, drop='first')
categorical_vars = churn_df.select_dtypes(include=['object', 'category']).columns.tolist()
encoded_features = encoder.fit_transform(churn_df[categorical_vars])
encoded_feature_names = encoder.get_feature_names_out(categorical_vars)

encoded_df = pd.DataFrame(encoded_features, columns=encoded_feature_names, index=churn_df.index)

churn_df_encoded = churn_df.drop(columns=categorical_vars).join(encoded_df)


In [5]:
X = churn_df_encoded.drop(columns=['churn'])
y = churn_df_encoded['churn']

scaler = StandardScaler().set_output(transform='pandas')
X_scaled = scaler.fit_transform(X)

features_scaled = pd.DataFrame(X_scaled, columns=X.columns, index=churn_df.index)

X_train, X_test, y_train, y_test = train_test_split(features_scaled, y, test_size=0.2, random_state=42)

logreg = LogisticRegression()
logreg.fit(X_train, y_train)
logreg_pred = logreg.predict(X_test)

logreg_pred_series = pd.Series(logreg_pred)
print(logreg_pred_series.value_counts())
print()

rf = RandomForestClassifier()
rf.fit(X_train, y_train)
rf_pred = rf.predict(X_test)

rf_pred_series = pd.Series(rf_pred)
print(rf_pred_series.value_counts())

0    1165
1     135
dtype: int64

0    1297
1       3
dtype: int64


In [7]:
logreg_cr = classification_report(y_test, logreg_pred)
logreg_cm = confusion_matrix(y_test, logreg_pred)

rf_cr = classification_report(y_test, rf_pred)
rf_cm = confusion_matrix(y_test, rf_pred)

logreg_accuracy = logreg_cm.trace() / logreg_cm.sum()
rf_accuracy = rf_cm.trace() / rf_cm.sum()

higher_accuracy = "LogisticRegression" if logreg_accuracy > rf_accuracy else "RandomForest"
print(higher_accuracy)

RandomForest
